In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

import requests, shutil, os, sys
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display
import numpy as np
import copy

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])#,
     #transforms.Normalize((0.5,), (0.5,))])

batch_size = 1

trainset = torchvision.datasets.EMNIST(root='/content', train=True,split="byclass",
                                        download=True, transform=transform)

testset = torchvision.datasets.EMNIST(root='/content', train=False,split="byclass",
                                       download=True, transform=transform)

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting /content/EMNIST/raw/gzip.zip to /content/EMNIST/raw


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning:

The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)



In [ ]:
trainset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
def read_img(path):
    return copy.deepcopy(mpimg.imread(path))

def save_img(path, image):
    mpimg.imsave(path, image)


def mono2rgb(img):
    if len(img.shape) == 3:
        return img
    return np.stack((img,img,img),axis=-1)



def show_img(lis,titles = ()):
    fig = make_subplots(
        rows=1, cols=len(lis),
        subplot_titles=(titles)
    )
    for i in range(len(lis)):
        fig.add_trace(px.imshow(lis[i]).data[0],row = 1, col = i+1)
    
    fig.show()

In [ ]:
ord("G")-55

16

In [ ]:
ord("0")-48

0

In [ ]:
ord('l')-62

46

In [ ]:
len(testset)

116323

In [ ]:
cnt = {}
for img,label in tqdm(trainset):
    if label not in cnt.keys():
        cnt[label] = 0
    cnt[label] += 1

In [ ]:
cnt

In [ ]:
dic = {}
for i in range(ord('0'),ord("9")+1):
    print(chr(i),i-48)
    dic[i-48] = chr(i)

for i in range(ord('A'),ord("Z")+1):
    print(chr(i),i-55)
    dic[i-55] = chr(i)

for i in range(ord('a'),ord("z")+1):
    print(chr(i),i-61)
    dic[i-61] = chr(i)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
A 10
B 11
C 12
D 13
E 14
F 15
G 16
H 17
I 18
J 19
K 20
L 21
M 22
N 23
O 24
P 25
Q 26
R 27
S 28
T 29
U 30
V 31
W 32
X 33
Y 34
Z 35
a 36
b 37
c 38
d 39
e 40
f 41
g 42
h 43
i 44
j 45
k 46
l 47
m 48
n 49
o 50
p 51
q 52
r 53
s 54
t 55
u 56
v 57
w 58
x 59
y 60
z 61


In [ ]:
tar = 48

In [ ]:
i = 0
img,label = trainset[i]
while label != tar:
    i += 1
    img,label = trainset[i]

img,label = trainset[i]
print(label)
show_img([mono2rgb(img[0].transpose(1,0))],titles=('Threshold Image',))

48


In [ ]:
i += 1
img,label = trainset[i]
while label != tar:
    i += 1
    img,label = trainset[i]

img,label = trainset[i]
print(label)
show_img([mono2rgb(img[0].transpose(1,0))],titles=('Threshold Image',))

48


In [ ]:
i += 1
img,label = trainset[i]
print(dic[label])
show_img([mono2rgb(img[0].transpose(1,0))],titles=('Threshold Image',))

e


In [ ]:
img[0].shape

torch.Size([28, 28])

In [ ]:
trainset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
from pathlib import Path

In [ ]:
save_path = Path("/content/emnist")
if not os.path.isdir(save_path):
        os.mkdir(save_path)
        
sub_folder = 'capital'
if not os.path.isdir(save_path/sub_folder):
        os.mkdir(save_path/sub_folder)

In [ ]:
dataset = []
for t in tqdm(trainset):
    if t[1] > 9 and t[1] <36:
        dataset.append(t) 

100%|██████████| 697932/697932 [00:59<00:00, 11642.20it/s]


In [ ]:
len(dataset)

188958

In [25]:
for i in tqdm(range(len(dataset))):
    img,label = dataset[i]
    img = img[0].transpose(1,0)
    img = mono2rgb(img)
    img = cv2.resize(img, (384,384))
    img = (1.0*(img>0.3))

    img = 1-img

    closing_SE_size = 3
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(closing_SE_size,closing_SE_size))
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel = kernel)

    kernel_size = 3
    img = cv2.erode(img,kernel =  np.ones((kernel_size,kernel_size),np.float32) ,iterations = 1)

    img = 1-img
    img = img.astype(np.float32)

    file_name = f"{dic[label]}__{i:06}.png"

    if not os.path.isdir(save_path/sub_folder):
        os.mkdir(save_path/sub_folder)
    
    save_img(save_path/sub_folder/file_name,img)
    

show_img([mono2rgb(img)])

Output hidden; open in https://colab.research.google.com to view.

In [26]:
output_filename = "/content/drive/MyDrive/Colab Notebooks/pathshala/sorted/english/capital/emnist_capital"
dir_name = "/content/emnist/capital"

In [27]:
shutil.make_archive(output_filename, 'zip', dir_name)

'/content/drive/MyDrive/Colab Notebooks/pathshala/sorted/english/capital/emnist_capital.zip'

In [28]:
len(os.listdir("/content/emnist/capital"))

188958

In [29]:
show_img([mono2rgb(img)])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
i += 1
img,label = dataset[i]
img = img[0].transpose(1,0)
img = mono2rgb(img)
img = cv2.resize(img, (384,384))
img = (1.0*(img>0.3))

closing_SE_size = 35
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(closing_SE_size,closing_SE_size))
img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel = kernel)
img = img.astype(np.float32)

In [ ]:
show_img([mono2rgb(img)])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img,label = dataset[i]
img = img[0].transpose(1,0)
img = mono2rgb(img)
img = cv2.resize(img, (384,384))
img = (1.0*(img>0.3))

closing_SE_size = 5
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(closing_SE_size,closing_SE_size))
img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel = kernel)
img = img.astype(np.float32)

In [ ]:
show_img([mono2rgb(img)])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

(numpy.ndarray, (1, 1))

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

(240000, 784)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
dic = {}
for i in range(ord('0'),ord("9")+1):
    print(chr(i),i-48)
    dic[i-48] = chr(i)

for i in range(ord('A'),ord("Z")+1):
    print(chr(i),i-55)
    dic[i-55] = chr(i)

for i in range(ord('a'),ord("b")+1):
    print(chr(i),i-61)
    dic[i-61] = chr(i)

for i in range(ord('d'),ord("h")+1):
    print(chr(i),i-62)
    dic[i-62] = chr(i)

print("n",43)
dic[43] = 'n'
print('q',44)
dic[44] = 'q'
print('r',45)
dic[45] = 'r'
print('t',46)
dic[46] = 't'

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
A 10
B 11
C 12
D 13
E 14
F 15
G 16
H 17
I 18
J 19
K 20
L 21
M 22
N 23
O 24
P 25
Q 26
R 27
S 28
T 29
U 30
V 31
W 32
X 33
Y 34
Z 35
a 36
b 37
d 38
e 39
f 40
g 41
h 42
n 43
q 44
r 45
t 46
